In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import solve, inv, norm

from cvxopt import matrix, solvers

from pystruct.inference import inference_dispatch

Пусть $y_i \in \{0,1,...,K-1\}$ для $i=1,2$. Найдем решение задачи дискретной оптимизации:

$$ \min_y \theta_1(y_1) + \theta_2(y_2) + \theta_{12}(y_1,y_2)$$

In [13]:
K = 5

np.random.seed(123)

theta_s = np.random.randn(K,)
theta_t = np.random.randn(K,)
theta_st = np.random.randn(K,K)

Наивное решение: полный перебор всех $K^2$ конфигураций переменных $y = (y_1, y_2)$.

In [14]:
f_min = np.inf

for i in range(K):
    for j in range(K):
        f = theta_s[i] + theta_t[j] + theta_st[i,j]
        if f < f_min:
            f_min = f
            y_sol = [i, j]
            
print('Solution:', y_sol)

Solution: [4, 1]


LP-релаксация:

\begin{gather}
\underset{\mu}{\text{min}} & \theta_1^T\mu_1 + \theta_2^T\mu_2 + \theta_{12}^T\mu_{12} \\
\text{s.t.} & 1^T\mu_1 = 1, 1^T\mu_2 = 1 \\
& M_{\text{vert}}\mu_{12} = \mu_2, M_{\text{horz}}\mu_{12} = \mu_1 \\
& \mu \geq 0
\end{gather}

Этот метод реализован в [pystruct.inference.inference_lp](https://pystruct.github.io/generated/pystruct.inference.inference_lp.html) (очень медленный).

In [15]:
I = np.eye(K)
O = np.zeros((K,K))
ones = np.ones((1,K))
zeros = np.zeros((1,K))

M_vert = np.kron(I, ones)
M_horz = np.kron(ones, I)

In [16]:
theta = np.hstack((theta_s, theta_t, theta_st.T.flatten())) # transp !

In [17]:
c = theta


G = np.block([[O, -I, M_vert],
              [-I, O, M_horz],
              [ones, zeros, np.zeros((1,K**2))],
              [zeros, ones, np.zeros((1,K**2))]])
h = np.vstack((np.zeros((2*K,1)), np.ones((2,1))))   

A = -np.eye(K+K+K**2)
b = np.zeros((K+K+K**2,1))

# objective
c = matrix(c)
# inequalities
A = matrix(A)
b = matrix(b)
# equalities
G = matrix(G)
h = matrix(h)


sol=solvers.lp(c, A, b, G, h, solver='glpk')
#print(sol)
xx = np.array(sol['x'])
x1 = xx[:K] # mu_1
x2 = xx[K:2*K] # mu_2

idx1 = np.argmax(x1)
idx2 = np.argmax(x2)

print('Solution:', [idx1, idx2])

Solution: [4, 1]


Динамическое программирование:

Целевая функция соответствует графу, состоящему из 2-х вершин, соединенных ребром.

<img src="http://apprize.info/data/nosql/nosql.files/image147.jpg" width=200>

Граф является _ациклическим_ (дерево), поэтому точное решение можно найти с помощью динамического программирования (в этом случае LP-релаксация также дает точное решение). Исходная задача сводится к поиску пути минимальной стоимости в графе вида (multistage graph):

<img src="https://www.researchgate.net/profile/Bartosz_Musznicki/publication/265412778/figure/fig1/AS:392039154896896@1470480826291/A-general-structure-of-a-multi-stage-graph.png" width=500>


In [18]:
n_nodes = 2
n_edges = 1
n_states = K

unary_potentials = np.zeros((n_nodes, n_states))
pairwise_potentials = np.zeros((n_edges, n_states, n_states))
edges = np.zeros((n_edges, 2), dtype=np.int32) # int

unary_potentials[0,:] = theta_s
unary_potentials[1,:] = theta_t
pairwise_potentials[0,:,:] = theta_st
edges[0,0] = 0
edges[0,1] = 1

# Find the maximizing assignment of a pairwise discrete energy function
y = inference_dispatch(-1.0*unary_potentials, -1.0*pairwise_potentials, edges, inference_method='max-product')

print('Solution:', y)

Solution: [4 1]
